<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/21_06_2024_timeline__dashboard_api_lineid_stoppoint_ewt_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Step 1: Get user inputs for line_id, stop_point_id, and direction
line_id = input('Enter the line id: ')
stop_point_id = input('Enter the stop point id: ')
direction = input('Enter the direction (inbound/outbound): ')
nbph = int(input('Enter the number of buses scheduled per hour: '))

# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Function to convert time to BST
def convert_to_bst(dt):
    # Define UTC and BST timezones
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    # Convert input datetime to UTC timezone
    utc_dt = utc_tz.localize(dt)

    # Convert UTC time to BST
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"

    params = {'direction': direction}

    response = requests.get(base_url, params=params)
    data = response.json()

    # Step 5: Parse the data
    predictions = []
    for item in data:
        arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
        arrival_time_bst = arrival_time + pd.Timedelta(hours=1)  # Convert GMT to BST
        predictions.append({
            'Line': item['lineName'],
            'Vehicle ID': item['vehicleId'],
            'Stop Point': stop_point_id,
            'Direction': direction,
            'Expected Arrival (BST)': arrival_time_bst,
            'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
        })

    # Step 6: Display the data in a readable form using pandas
    df = pd.DataFrame(predictions)

    # Sort by 'Expected Arrival (BST)'
    df = df.sort_values(by='Expected Arrival (BST)', ascending=True)

    # Convert 'Expected Arrival (HM)' to datetime format
    df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')

    # Calculate headway (considering only hour and minute)
    df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60

    # Calculate AWT (Average Waiting Time) as half of headway
    df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)

    # Calculate WAWT as the product of headway and AWT/bus
    df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)

    # Format 'Expected Arrival (BST)' column to string for better readability, ignoring seconds and microseconds
    df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

    return df

# Function to create the dynamic timeline dashboard
def create_dashboard(df):
    current_time_utc = datetime.utcnow()  # Get current UTC time
    current_time_bst = convert_to_bst(current_time_utc)  # Convert to BST

    # Create a timeline figure
    fig = go.Figure()

    # Add a big square for the stop point at the right end
    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=df['Stop Point'].iloc[0],  # Display stop point value
        hoverinfo='text'
    ))

    # Add label for the stop point value above the square
    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{df['Stop Point'].iloc[0]}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=30  # Adjust the yshift to move the label above the square
    )

    # Add circles for each bus moving towards the stop point
    for i, row in df.iterrows():
        expected_arrival = row['Expected Arrival (BST)']
        fig.add_trace(go.Scatter(
            x=[expected_arrival],
            y=[0],  # Y position of the stop point (fixed at 0)
            mode='markers+text',
            marker=dict(size=12, color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]),
            text=row['Vehicle ID'],
            textposition='top center',
            name=row['Vehicle ID'],
            hovertext=f'Expected Arrival: {expected_arrival}',
            hoverinfo='text'
        ))

    # Update layout
    fig.update_layout(
        title='Bus Positions by Expected Arrival Time',
        xaxis=dict(title='Time (BST)'),
        yaxis=dict(showticklabels=False),  # Remove y-axis tick labels
        showlegend=True
    )

    return fig

# Main loop to fetch and display data
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)

    # Calculate total sum of WAWT
    total_wawt = df['WAWT'].sum()

    # Calculate time difference between first and last observed bus
    min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
    max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
    time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60

    # Calculate AWT
    awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0

    # Calculate SWT
    swt = round(60 / (nbph * 2), 2)

    # Calculate EWT
    ewt = round(awt - swt, 2)

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses(minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
        'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
    })

    clear_output(wait=True)  # Clear previous output to update the table and dashboard

    # Display the updated predictions
    print("Arrival Predictions:")
    display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])

    # Display the summary metrics
    print("\nSummary Metrics:")
    display(summary_df)

    # Create and display the dynamic timeline dashboard
    fig = create_dashboard(df)
    fig.show()

    time.sleep(15)  # Wait for 15 seconds before the next update


Arrival Predictions:


,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
0,214,LA19JZW,490008660N,inbound,14:06:00,0.0,0.0,0.0
2,214,LA19KBJ,490008660N,inbound,14:16:00,10.0,5.0,50.0
1,214,LA19KAU,490008660N,inbound,14:22:00,6.0,3.0,18.0
3,214,LA19KCE,490008660N,inbound,14:28:00,6.0,3.0,18.0



Summary Metrics:


,Metric,Value
0,Number of buses scheduled per hour (nbph),6.00
1,Total WAWT (minutes),86.00
2,Time difference between 1st and last observed ...,22.00
3,AWT (minutes),3.91
4,SWT (minutes),5.00
5,EWT (minutes),-1.09


KeyboardInterrupt: 

In [ ]:
# Step 1: Get user inputs for line_id, stop_point_id, and direction
line_id = input('Enter the line id: ')
stop_point_id = input('Enter the stop point id: ')
direction = input('Enter the direction (inbound/outbound): ')
nbph = int(input('Enter the number of buses scheduled per hour: '))

# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Function to convert time to BST
def convert_to_bst(dt):
    # Define UTC and BST timezones
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    # Convert input datetime to UTC timezone
    utc_dt = utc_tz.localize(dt)

    # Convert UTC time to BST
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"

        params = {'direction': direction}

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        data = response.json()

        # Step 5: Parse the data
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + pd.Timedelta(hours=1)  # Convert GMT to BST
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
            })

        # Step 6: Display the data in a readable form using pandas
        df = pd.DataFrame(predictions)

        # Sort by 'Expected Arrival (BST)'
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)

        # Convert 'Expected Arrival (HM)' to datetime format
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')

        # Calculate headway (considering only hour and minute)
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60

        # Calculate AWT (Average Waiting Time) as half of headway
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)

        # Calculate WAWT as the product of headway and AWT/bus
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)

        # Format 'Expected Arrival (BST)' column to string for better readability, ignoring seconds and microseconds
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None  # Return None if there's an error

# Function to create the dynamic timeline dashboard
def create_dashboard(df):
    current_time_utc = datetime.utcnow()  # Get current UTC time
    current_time_bst = convert_to_bst(current_time_utc)  # Convert to BST

    # Create a timeline figure
    fig = go.Figure()

    # Add a big square for the stop point at the right end
    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=df['Stop Point'].iloc[0],  # Display stop point value
        hoverinfo='text'
    ))

    # Add label for the stop point value above the square
    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{df['Stop Point'].iloc[0]}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=30  # Adjust the yshift to move the label above the square
    )

    # Add circles for each bus moving towards the stop point
    for i, row in df.iterrows():
        expected_arrival = row['Expected Arrival (BST)']
        fig.add_trace(go.Scatter(
            x=[expected_arrival],
            y=[0],  # Y position of the stop point (fixed at 0)
            mode='markers+text',
            marker=dict(size=12, color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]),
            text=row['Vehicle ID'],
            textposition='top center',
            name=row['Vehicle ID'],
            hovertext=f'Expected Arrival: {expected_arrival}',
            hoverinfo='text'
        ))

    # Update layout
    fig.update_layout(
        title='Bus Positions by Expected Arrival Time',
        xaxis=dict(title='Time (BST)'),
        yaxis=dict(showticklabels=False),  # Remove y-axis tick labels
        showlegend=True
    )

    return fig

# Main loop to fetch and display data
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)

    if df is not None:  # Check if data fetching was successful
        clear_output(wait=True)  # Clear previous output to update the table and dashboard

        # Calculate summary metrics
        total_wawt = df['WAWT'].sum()

        min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
        max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
        time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60

        awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
        swt = round(60 / (nbph * 2), 2)
        ewt = round(awt - swt, 2)

        # Create summary DataFrame
        summary_df = pd.DataFrame({
            'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
            'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
        })

        # Display the updated predictions
        print("Arrival Predictions:")
        display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])

        # Display the summary metrics
        print("\nSummary Metrics:")
        display(summary_df)

        # Create and display the dynamic timeline dashboard
        fig = create_dashboard(df)
        fig.show()

    else:
        # Display an error message if data fetching failed
        print("Failed to fetch data. Please check your inputs or try again later.")

    time.sleep(15)  # Wait for 15 seconds before the next update


Arrival Predictions:


<ipython-input-11-841c3a20c49d>:154: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-11-841c3a20c49d>:155: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
2,22,YY66OZR,490012375E,outbound,14:48:00,0.0,0.0,0.0
1,22,YY66OZP,490012375E,outbound,15:01:00,13.0,6.5,84.5
0,22,BV66VJP,490012375E,outbound,15:07:00,6.0,3.0,18.0



Summary Metrics:


,Metric,Value
0,Number of buses scheduled per hour (nbph),6.00
1,Total WAWT (minutes),102.50
2,Time difference between 1st and last observed ...,19.00
3,AWT (minutes),5.39
4,SWT (minutes),5.00
5,EWT (minutes),0.39
